In [1]:
import ROOT
from ROOT import TFile, TTree, TObject, TCanvas
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

from ROOT import TGraph, gRandom

import pandas as pd
from root_pandas import read_root, to_root

import time
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

import uproot

Welcome to JupyROOT 6.24/02


In [2]:
#work with other files, should
#just load individual files as df_ef

In [3]:
depo_dir = '/home/steve/Desktop/ee/'
root_f = "ee_ttbarsignalplustau_fromDilepton_TuneCP5.root"#load your own files here

In [4]:
data = uproot.open(depo_dir + root_f, key = "ttBar_treeVariables_step8;3")

In [5]:
data["ttBar_treeVariables_step8;3"].keys()

['top_pt',
 'top_phi',
 'top_rapidity',
 'top_arapidity',
 'top_eta',
 'top_mass',
 'top_boosted_theta',
 'top_boosted_eta',
 'top_theta_v2',
 'tbar_pt',
 'tbar_phi',
 'tbar_rapidity',
 'tbar_arapidity',
 'tbar_eta',
 'tbar_mass',
 'tbar_boosted_theta',
 'tbar_boosted_eta',
 'l_pt',
 'lbar_pt',
 'e_pt',
 'ebar_pt',
 'mu_pt',
 'mubar_pt',
 'b_pt',
 'bbar_pt',
 'nu_pt',
 'nubar_pt',
 'l_eta',
 'lbar_eta',
 'e_eta',
 'ebar_eta',
 'mu_eta',
 'mubar_eta',
 'b_eta',
 'bbar_eta',
 'nu_eta',
 'nubar_eta',
 'l_phi',
 'lbar_phi',
 'e_phi',
 'ebar_phi',
 'mu_phi',
 'mubar_phi',
 'b_phi',
 'bbar_phi',
 'nu_phi',
 'nubar_phi',
 'met_pt',
 'ttbar_pt',
 'ttbar_phi',
 'ttbar_rapidity',
 'ttbar_arapidity',
 'ttbar_delta_phi',
 'ttbar_delta_eta',
 'ttbar_delta_rapidity',
 'ttbar_mass',
 'ttbar_eta',
 'ttbar_boosted_pt',
 'llbar_pt',
 'llbar_phi',
 'llbar_rapidity',
 'llbar_arapidity',
 'llbar_delta_phi',
 'llbar_delta_eta',
 'llbar_delta_rapidity',
 'llbar_mass',
 'bbbar_pt',
 'bbbar_phi',
 'bbbar_rapid

### try same thing with np array

In [6]:
l_eta = data["ttBar_treeVariables_step8;3"+'/l_eta'].array(library="np")
lbar_eta = data["ttBar_treeVariables_step8;3"+'/lbar_eta'].array(library="np")

l_phi = data["ttBar_treeVariables_step8;3"+'/l_phi'].array(library="np")
lbar_phi = data["ttBar_treeVariables_step8;3"+'/lbar_phi'].array(library="np")

In [7]:
b_eta = data["ttBar_treeVariables_step8;3"+'/b_eta'].array(library="np")
bbar_eta = data["ttBar_treeVariables_step8;3"+'/bbar_eta'].array(library="np")

b_phi = data["ttBar_treeVariables_step8;3"+'/b_phi'].array(library="np")
bbar_phi = data["ttBar_treeVariables_step8;3"+'/bbar_phi'].array(library="np")

In [8]:
nu_eta = data["ttBar_treeVariables_step8;3"+'/nu_eta'].array(library="np")
nubar_eta = data["ttBar_treeVariables_step8;3"+'/nubar_eta'].array(library="np")

nu_phi = data["ttBar_treeVariables_step8;3"+'/nu_phi'].array(library="np")
nubar_phi = data["ttBar_treeVariables_step8;3"+'/nubar_phi'].array(library="np")

In [9]:
lb_delta_eta = np.abs(np.subtract(l_eta,b_eta))
lbbar_delta_eta = np.abs(np.subtract(l_eta,bbar_eta))
lnu_delta_eta = np.abs(np.subtract(l_eta,nu_eta))
lnubar_delta_eta = np.abs(np.subtract(l_eta,nubar_eta))
lbarb_delta_eta = np.abs(np.subtract(lbar_eta,b_eta))
lbarbbar_delta_eta = np.abs(np.subtract(lbar_eta,bbar_eta))
lbarnu_delta_eta = np.abs(np.subtract(lbar_eta,nu_eta))
lbarnubar_delta_eta = np.abs(np.subtract(lbar_eta,nubar_eta))
bnu_delta_eta = np.abs(np.subtract(b_eta,nu_eta))
bnubar_delta_eta = np.abs(np.subtract(b_eta,nubar_eta))
bbarnu_delta_eta = np.abs(np.subtract(bbar_eta,nu_eta))
bbarnubar_delta_eta = np.abs(np.subtract(bbar_eta,nubar_eta))

In [10]:
df = pd.DataFrame()

In [11]:
df['lb_delta_eta'] = lb_delta_eta.tolist()
df['lbbar_delta_eta'] = lbbar_delta_eta.tolist()
df['lnu_delta_eta'] = lnu_delta_eta.tolist()
df['lnubar_delta_eta'] = lnubar_delta_eta.tolist()
df['lbarb_delta_eta'] = lbarb_delta_eta.tolist()
df['lbarbbar_delta_eta'] = lbarbbar_delta_eta.tolist()
df['lbarnu_delta_eta'] = lbarnu_delta_eta.tolist()
df['lbarnubar_delta_eta'] = lbarnubar_delta_eta.tolist()
df['bnu_delta_eta'] = bnu_delta_eta.tolist()
df['bnubar_delta_eta'] = bnubar_delta_eta.tolist()
df['bbarnu_delta_eta'] = bbarnu_delta_eta.tolist()
df['bbarnubar_delta_eta'] = bbarnubar_delta_eta.tolist()

In [12]:
df

,lb_delta_eta,lbbar_delta_eta,lnu_delta_eta,lnubar_delta_eta,lbarb_delta_eta,lbarbbar_delta_eta,lbarnu_delta_eta,lbarnubar_delta_eta,bnu_delta_eta,bnubar_delta_eta,bbarnu_delta_eta,bbarnubar_delta_eta
0,0.196395,0.401817,1.523488,1.100096,0.437924,0.232502,0.889169,0.465777,1.327093,0.903701,1.121671,0.698279
1,0.172882,0.338345,0.095944,2.081524,0.151838,0.663065,0.420664,1.756804,0.268826,1.908642,0.242400,2.419869
2,0.927498,0.191133,3.020247,0.361787,1.366539,2.485170,0.726209,1.932250,2.092748,0.565711,3.211379,0.552920
3,0.180216,0.155463,1.146340,1.088407,1.069548,1.094300,0.103424,2.338171,0.966124,1.268623,0.990877,1.243870
4,1.693352,0.851445,0.393477,0.611020,1.780469,0.764327,0.480594,0.698138,1.299875,1.082331,1.244921,1.462465
...,...,...,...,...,...,...,...,...,...,...,...,...
97717,0.706968,0.430491,0.675099,0.992337,1.965102,0.827643,0.583036,0.265797,1.382067,1.699306,0.244608,0.561846
97718,0.612008,0.811103,1.935039,0.069588,0.457409,1.880519,0.865623,0.999828,1.323032,0.542420,2.746142,0.880691
97719,0.075397,2.549377,1.135454,0.695678,1.313474,1.160506,0.253417,0.693193,1.060057,0.620281,1.413923,1.853699
97720,1.103137,0.719776,0.035610,0.183387,0.379309,0.004053,0.759438,0.907215,1.138747,1.286524,0.755386,0.903163


In [13]:
#phi
lb_delta_phi = np.abs(np.subtract(l_phi,b_phi))
lbbar_delta_phi = np.abs(np.subtract(l_phi,bbar_phi))
lnu_delta_phi = np.abs(np.subtract(l_phi,nu_phi))
lnubar_delta_phi = np.abs(np.subtract(l_phi,nubar_phi))
lbarb_delta_phi = np.abs(np.subtract(lbar_phi,b_phi))
lbarbbar_delta_phi = np.abs(np.subtract(lbar_phi,bbar_phi))
lbarnu_delta_phi = np.abs(np.subtract(lbar_phi,nu_phi))
lbarnubar_delta_phi = np.abs(np.subtract(lbar_phi,nubar_phi))
bnu_delta_phi = np.abs(np.subtract(b_phi,nu_phi))
bnubar_delta_phi = np.abs(np.subtract(b_phi,nubar_phi))
bbarnu_delta_phi = np.abs(np.subtract(bbar_phi,nu_phi))
bbarnubar_delta_phi = np.abs(np.subtract(bbar_phi,nubar_phi))

In [14]:
df['lb_delta_phi'] = lb_delta_phi.tolist()
df['lbbar_delta_phi'] = lbbar_delta_phi.tolist()
df['lnu_delta_phi'] = lnu_delta_phi.tolist()
df['lnubar_delta_phi'] = lnubar_delta_phi.tolist()
df['lbarb_delta_phi'] = lbarb_delta_phi.tolist()
df['lbarbbar_delta_phi'] = lbarbbar_delta_phi.tolist()
df['lbarnu_delta_phi'] = lbarnu_delta_phi.tolist()
df['lbarnubar_delta_phi'] = lbarnubar_delta_phi.tolist()
df['bnu_delta_phi'] = bnu_delta_phi.tolist()
df['bnubar_delta_phi'] = bnubar_delta_phi.tolist()
df['bbarnu_delta_phi'] = bbarnu_delta_phi.tolist()
df['bbarnubar_delta_phi'] = bbarnubar_delta_phi.tolist()

In [15]:
df

,lb_delta_eta,lbbar_delta_eta,lnu_delta_eta,lnubar_delta_eta,lbarb_delta_eta,lbarbbar_delta_eta,lbarnu_delta_eta,lbarnubar_delta_eta,bnu_delta_eta,bnubar_delta_eta,...,lnu_delta_phi,lnubar_delta_phi,lbarb_delta_phi,lbarbbar_delta_phi,lbarnu_delta_phi,lbarnubar_delta_phi,bnu_delta_phi,bnubar_delta_phi,bbarnu_delta_phi,bbarnubar_delta_phi
0,0.196395,0.401817,1.523488,1.100096,0.437924,0.232502,0.889169,0.465777,1.327093,0.903701,...,3.664300,5.247219,1.886725,2.452430,0.215773,1.367145,2.102498,0.519580,2.236657,3.819575
1,0.172882,0.338345,0.095944,2.081524,0.151838,0.663065,0.420664,1.756804,0.268826,1.908642,...,2.616607,2.842055,2.508073,2.777720,1.292706,1.518155,3.800779,4.026227,1.485014,1.259565
2,0.927498,0.191133,3.020247,0.361787,1.366539,2.485170,0.726209,1.932250,2.092748,0.565711,...,1.233505,1.784331,0.154567,5.119201,2.629722,3.180547,2.475155,3.025981,2.489479,1.938653
3,0.180216,0.155463,1.146340,1.088407,1.069548,1.094300,0.103424,2.338171,0.966124,1.268623,...,2.495975,3.301652,0.976984,3.188871,2.601468,3.407145,3.578452,4.384129,0.587404,0.218274
4,1.693352,0.851445,0.393477,0.611020,1.780469,0.764327,0.480594,0.698138,1.299875,1.082331,...,3.837617,4.687695,0.464101,1.338921,0.758297,1.608375,1.222399,2.072476,2.097218,2.947296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97717,0.706968,0.430491,0.675099,0.992337,1.965102,0.827643,0.583036,0.265797,1.382067,1.699306,...,3.310601,3.084955,0.235160,1.940516,4.381040,4.155393,4.616199,4.390553,2.440524,2.214877
97718,0.612008,0.811103,1.935039,0.069588,0.457409,1.880519,0.865623,0.999828,1.323032,0.542420,...,2.838812,0.561753,1.288679,0.758161,0.401717,2.998848,0.886962,4.287527,1.159878,2.240686
97719,0.075397,2.549377,1.135454,0.695678,1.313474,1.160506,0.253417,0.693193,1.060057,0.620281,...,1.062148,2.142953,1.686344,1.356847,1.921451,3.002256,3.607795,4.688600,3.278298,4.359103
97720,1.103137,0.719776,0.035610,0.183387,0.379309,0.004053,0.759438,0.907215,1.138747,1.286524,...,4.348174,1.368332,5.126985,2.493909,0.055158,2.924685,5.182143,2.202300,2.549067,0.430775


In [16]:
df.keys()

Index(['lb_delta_eta', 'lbbar_delta_eta', 'lnu_delta_eta', 'lnubar_delta_eta',
       'lbarb_delta_eta', 'lbarbbar_delta_eta', 'lbarnu_delta_eta',
       'lbarnubar_delta_eta', 'bnu_delta_eta', 'bnubar_delta_eta',
       'bbarnu_delta_eta', 'bbarnubar_delta_eta', 'lb_delta_phi',
       'lbbar_delta_phi', 'lnu_delta_phi', 'lnubar_delta_phi',
       'lbarb_delta_phi', 'lbarbbar_delta_phi', 'lbarnu_delta_phi',
       'lbarnubar_delta_phi', 'bnu_delta_phi', 'bnubar_delta_phi',
       'bbarnu_delta_phi', 'bbarnubar_delta_phi'],
      dtype='object')

## old code

In [19]:
df.to_root('/home/steve/Desktop/test_root_file/testing_tree_creation_v0.0.2.root', key = "DecayTree_step8;3")

## load data

In [17]:
depo_dir_n = '/home/steve/Desktop/test_root_file/'
root_f_n = "testing_tree_creation_v0.0.2.root"#load your own files here

In [18]:
data_n = uproot.open(depo_dir_n + root_f_n, key = "DecayTree_step8;3")

In [25]:
data_n["DecayTree_step8;3;1"].keys()#why simicol +1?

['lb_delta_eta',
 'lbbar_delta_eta',
 'lnu_delta_eta',
 'lnubar_delta_eta',
 'lbarb_delta_eta',
 'lbarbbar_delta_eta',
 'lbarnu_delta_eta',
 'lbarnubar_delta_eta',
 'bnu_delta_eta',
 'bnubar_delta_eta',
 'bbarnu_delta_eta',
 'bbarnubar_delta_eta',
 'lb_delta_phi',
 'lbbar_delta_phi',
 'lnu_delta_phi',
 'lnubar_delta_phi',
 'lbarb_delta_phi',
 'lbarbbar_delta_phi',
 'lbarnu_delta_phi',
 'lbarnubar_delta_phi',
 'bnu_delta_phi',
 'bnubar_delta_phi',
 'bbarnu_delta_phi',
 'bbarnubar_delta_phi',
 '__index__']